In [6]:
import pandas as pd 
import plotly.express as px 
import warnings 
from internproject.constants import ROOT_DATA_DIR

add_lp = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_add_lp.parquet")
remove_lp = pd.read_parquet(
    ROOT_DATA_DIR / "stETH_ETH_remove_lp.parquet"
)
lp_transfers = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_lp_transfers.parquet")

In [8]:
stETH_ETH_apr_df = pd.read_csv(
    ROOT_DATA_DIR
    / 'daily_snapshots/0xDC24316b9AE028F1497c275EB9192a3Ea0f67022_apr_df.csv',
    index_col=0,
    parse_dates=True,
).dropna()

##### Reward rate data

In [11]:
stETH_ETH_incentive_rewards_df = pd.read_parquet(
    ROOT_DATA_DIR / 'raw_pool_data/curve_stETH_ETH/extra_rewards_df.parquet'
)
# quantity of LDO rewards issues per second 
px.line(stETH_ETH_incentive_rewards_df['reward_rate'])

### Individual behavior analysis of LARGEST WHALE

In [13]:
null_address = '0x0000000000000000000000000000000000000000'
add_liquidity_transfers = lp_transfers[lp_transfers["from"] == null_address]
withdraw_liquidity_transfers = lp_transfers[lp_transfers["to"] == null_address]

In [16]:
withdraw_liquidity_transfers['value'] = withdraw_liquidity_transfers['value']* -1

warnings.filterwarnings('ignore')
withdraw_liquidity_transfers['value']

timestamp
2021-01-05 17:12:33     -2.000945
2021-01-05 17:46:38   -499.501433
2021-01-05 21:03:54     -0.999754
2021-01-06 03:36:23    -10.037649
2021-01-06 12:17:18     -1.002055
                          ...    
2023-06-20 22:25:35     -0.000000
2023-06-20 22:33:11    -65.033669
2023-06-20 22:40:35    -51.167100
2023-06-20 23:28:11     -0.000000
2023-06-21 00:00:11     -0.003230
Name: value, Length: 27851, dtype: float64

In [17]:
add_liquidity_transfers['user'] = add_liquidity_transfers['to']
withdraw_liquidity_transfers['user'] = withdraw_liquidity_transfers['from']

transfer_df = pd.concat([add_liquidity_transfers, withdraw_liquidity_transfers])
transfer_df.columns

warnings.filterwarnings('ignore')

In [23]:
# finding cumulative sum 
transfer_df.sort_index(inplace=True)
transfer_df['cumsol_col'] = transfer_df.groupby(['user'])['value'].cumsum()

# sorting from smallest to largest
whales_sorted_df = transfer_df.groupby('user').max().sort_values('cumsol_col')

In [22]:
largest_whale = transfer_df[transfer_df['user'] == '0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3']
largest_whale

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user,cumsol_col
timestamp,,,,,,,,,,,
2022-09-26 14:36:23,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118782.299389,Transfer,15618180,367,385,0x200853bc39a0b4ac57f3a396d22a39a070686ec5e63a...,2022-09-26 14:36:23,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118782.299389
2022-09-26 19:08:35,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0.921205,Transfer,15619528,39,152,0xe62463f53c97cc3542eb72887719022b33172209954b...,2022-09-26 19:08:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118783.220593
2022-09-27 04:24:35,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,1.909552,Transfer,15622291,162,246,0xfb072e072c6557451effe6a61158eed4360decd747c2...,2022-09-27 04:24:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118785.130146
2022-09-27 13:44:23,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,1.903760,Transfer,15625068,172,316,0x21ba8a9e96c6d327fb6df54ff59a683ed8dc418fa2c7...,2022-09-27 13:44:23,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118787.033906
2022-09-27 23:17:23,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,1.905536,Transfer,15627904,76,160,0xa1add66f1689c1ffc90c12e53dd7f4fde42a066ccea3...,2022-09-27 23:17:23,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118788.939442
...,...,...,...,...,...,...,...,...,...,...,...
2023-06-07 13:42:47,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428863,87,247,0xaf7747639152a7093b205d38308052f6fba17716c65c...,2023-06-07 13:42:47,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,107303.607420
2023-06-07 13:43:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428867,163,204,0x7183029355114ef096e371caae8dc304e9c21f78cecc...,2023-06-07 13:43:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,105303.607420
2023-06-07 13:43:59,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428869,70,133,0xfea33c7d1cf4a21e4269a595047bd551dbd8af65b5da...,2023-06-07 13:43:59,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,103303.607420


#### How many transactions did they take part in? 

In [34]:
added = add_liquidity_transfers[add_liquidity_transfers['to'] == '0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3']
len(added)

185

In [33]:
withdrew = withdraw_liquidity_transfers[withdraw_liquidity_transfers['from'] == '0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3']
len(withdrew)

14

In [36]:
# number of transactions this whale took part in
num_transactions = len(added) - len(withdrew) 
num_transactions

171

##### How long did this whale stay in the pool? 

In [40]:
whales_sorted_df

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,cumsol_col
user,,,,,,,,,,
0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0x0000000000000000000000000000000000000000,-80752.602372,Transfer,16038263,0,0,0x2166dcfa7f9a828635963de909c78fd3bee673bcbf13...,2022-11-24 07:11:23,-80752.602372
0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x0000000000000000000000000000000000000000,-19901.074656,Transfer,15966529,14,34,0xe56afad8c1557b19d87989037131406484c9577bde57...,2022-11-14 06:37:23,-19901.074656
0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0x0000000000000000000000000000000000000000,-13000.000000,Transfer,14615872,80,129,0xed4804f8260f468f8bf7dbe24c081e57fa9fe1f117f8...,2022-04-19 13:52:55,-13000.000000
0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x0000000000000000000000000000000000000000,-6457.421531,Transfer,14179288,42,73,0x321427338678b6b2eb4c4a3751cab14e15c763fec7b4...,2022-02-10 16:04:10,-6457.421531
0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x0000000000000000000000000000000000000000,-4604.795480,Transfer,17379760,73,131,0x44f3fdac141b11e692489ba91ecd070e3cc45139b0aa...,2023-05-31 15:31:47,-4604.795480
...,...,...,...,...,...,...,...,...,...,...
0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,86061.625846,Transfer,14937471,273,587,0xef05f74c37b9b32cbfdaf80933c076b344a00af86bb0...,2022-06-10 08:37:37,105327.329795
0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,60862.175792,Transfer,13188689,269,356,0xff8570fa3c8b4b43055ebc7bd984ab412e68a8295530...,2021-09-09 01:44:23,114142.222772
0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,114348.306873,Transfer,17512484,382,1100,0xf2c693113c6e9693eced243131d5e7dfe43c3148cf74...,2023-06-19 08:04:23,118845.280202


In [41]:
whales_sorted_df[whales_sorted_df['cumsol_col'] < 0]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,cumsol_col
user,,,,,,,,,,
0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0x0000000000000000000000000000000000000000,-80752.602372,Transfer,16038263,0,0,0x2166dcfa7f9a828635963de909c78fd3bee673bcbf13...,2022-11-24 07:11:23,-80752.602372
0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x0000000000000000000000000000000000000000,-19901.074656,Transfer,15966529,14,34,0xe56afad8c1557b19d87989037131406484c9577bde57...,2022-11-14 06:37:23,-19901.074656
0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0x0000000000000000000000000000000000000000,-13000.000000,Transfer,14615872,80,129,0xed4804f8260f468f8bf7dbe24c081e57fa9fe1f117f8...,2022-04-19 13:52:55,-13000.000000
0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x0000000000000000000000000000000000000000,-6457.421531,Transfer,14179288,42,73,0x321427338678b6b2eb4c4a3751cab14e15c763fec7b4...,2022-02-10 16:04:10,-6457.421531
0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x0000000000000000000000000000000000000000,-4604.795480,Transfer,17379760,73,131,0x44f3fdac141b11e692489ba91ecd070e3cc45139b0aa...,2023-05-31 15:31:47,-4604.795480
...,...,...,...,...,...,...,...,...,...,...
0xbC6433537D8125a72b849b1d777a7E0FfE3DdE02,0xbC6433537D8125a72b849b1d777a7E0FfE3DdE02,0x0000000000000000000000000000000000000000,-0.000523,Transfer,17468383,101,210,0xc50f9a6f6c75f21eb03d618f5493992de4fd5ce65b89...,2023-06-13 03:22:59,-0.000523
0x95B097D903993F54E1afd2D7E534F43d4dd32BD5,0x95B097D903993F54E1afd2D7E534F43d4dd32BD5,0x0000000000000000000000000000000000000000,-0.000009,Transfer,16911337,135,285,0xeeccbc0406c55d79980852f93aa84e221b9582fe4018...,2023-03-26 11:34:59,-0.000521
0xE20ee1D250C8d3e0BD140306121001BfDc7a1201,0xE20ee1D250C8d3e0BD140306121001BfDc7a1201,0x0000000000000000000000000000000000000000,-0.000100,Transfer,13898358,78,139,0xd9b0107373665e1166eebd3b95df6aaf98fe21bd93bd...,2021-12-29 05:34:32,-0.000100


##### There are 1,351 unique users who have a negative balance. We will consider them as not part of the pool anymore. 

In [57]:
# choose random user from this group of people who are not part of the pool anymore
person = '0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A'
add_liquidity_transfers[add_liquidity_transfers["to"] == person]


,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user
timestamp,,,,,,,,,,


In [58]:
withdraw_liquidity_transfers[withdraw_liquidity_transfers["from"] == person]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user
timestamp,,,,,,,,,,
2023-03-14 03:13:47,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A,0x0000000000000000000000000000000000000000,-0.000010,Transfer,16823446,53,161,0x358adf18c89be02cfa114e8a316b8d7159cf19429064...,2023-03-14 03:13:47,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A
2023-03-16 06:17:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A,0x0000000000000000000000000000000000000000,-0.000188,Transfer,16838583,111,276,0x440e9910c26cbd8920a05bd9afa6d79fceb65b97599c...,2023-03-16 06:17:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A
2023-03-16 06:49:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A,0x0000000000000000000000000000000000000000,-0.000188,Transfer,16838742,68,229,0x999da014b76f2c9238b9a8d4f145b2a2af9924c8f00a...,2023-03-16 06:49:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A
2023-03-16 09:07:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A,0x0000000000000000000000000000000000000000,-0.001323,Transfer,16839420,74,185,0x9bbb863e89c591eed89ebc8867202b5ddc3a89874317...,2023-03-16 09:07:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A
2023-03-28 02:07:47,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A,0x0000000000000000000000000000000000000000,-0.006572,Transfer,16922769,35,32,0xa2add4120a7a7685049c4800873d40f45fd55da0b9f6...,2023-03-28 02:07:47,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A


##### This doesn't make sense to me because this person ONLY withdrew from the pool and never added liquidity. I noticed this with multiple users with negative balances.

### Whale's balance over time compared to LDO reward rate 

In [59]:
# locating unique id of largest whale 
largest_whale = transfer_df[transfer_df['user'] == '0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3']
